# Librerias necesarias

In [103]:
from embeddings import *
from ollama import Client

# Configuración inicial

## Configuración embeddins

In [104]:
modelPath = "thenlper/gte-large"

model_kwargs = {'device':'cuda'}

encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

c:\Users\PC\miniconda3\envs\evariste-env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [105]:
db = cargar_embeddings('emmbeddings_db', 'indices_paises_wikis', embeddings)

## Configuración LLM

In [106]:
client = Client(host='http://localhost:3030')

# Preparación para preguntar

In [108]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

retriever = db.as_retriever()

system_prompt=("You are a highly intelligent question answering bot. "
               "If I ask you a question that is rooted in truth, you will give you the answer. "
               "If I ask you a question that is nonsense, trickery, or has no clear answer, you will respond with 'Unknown'. "
               "You will answer concisely. "
               "Use the given context as a support to answer the question if you can't answer the question."
               "Context: {context}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Answer the query.\n{format_instructions}\n{question}\n"),
    ],
)

ollama = Ollama(
    base_url='http://localhost:3030',
    model="llama2-uncensored"
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def instruction(*args):

    return ("Format the information as a table with columns for country name and country continent. "
            "Your response should be a table")

rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough(),
     "format_instructions": instruction}
    | prompt
    | ollama
    | StrOutputParser()
)

print(rag_chain.invoke('Give me the Name of the countries where Continent it is Asia or Continent it is Europe'))

| Country | Continent |
|-------------------|-------------|
| Afghanistan | Asia |
| Albania | Europe |
| Algeria | Africa |
| Andorra | Europe |
| Angola | Africa |
| Antigua and Barbuda | North America |
| Argentina | South America |
| Armenia | Asia |
| Australia | Oceania |
| Austria | Europe |
| Azerbaijan | Asia |
| Bahamas | North America |
| Bahrain | Asia |
| Bangladesh | Asia |
| Barbados | North America |
| Belarus | Europe |
| Belgium | Europe |
| Belize | Central America |
| Benin | Africa |
| Bhutan | Asia |
| Bolivia | South America |
| Bosnia and Herzegovina | Europe |
| Botswana | Africa |
| Brazil | South America |
| Brunei | Asia |
| Bulgaria | Europe |
| Burkina Faso | Africa |
| Burundi | Africa |
| Cambodia | Asia |
| Cameroon | Africa |
| Canada | North America |
| Cape Verde | Africa |
| Central African Republic | Africa |
| Chad | Africa |
| Chile | South America |
| China | Asia |
| Colombia | South America |
| Comoros | Africa |
| Congo | Africa |
| Costa Ric